IMPORTATIONS

In [8]:
pip install -U imbalanced-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
#IMPORTATIONS GENERALES
import csv
import numpy as np
import pandas as pd
import os
import nltk
import matplotlib.pyplot as plt
#SKLEARN 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix

   #LES 4 MODELES
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
#AUTRE
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

import seaborn as sns

# NLTK
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


AttributeError: partially initialized module 'nltk' has no attribute 'data' (most likely due to a circular import)

FONCTIONS ET DICO

In [ ]:

def preprocess_comma(file,newfile):
    data_frame = pd.read_csv(file)

    # Retirer les virgules sauf la première de chaque ligne
    data_frame.iloc[:, 7:] =  data_frame.iloc[:, 7:].replace(to_replace=',', value='', regex=True)
# Enregistrer dans un nouveau fichier CSV avec un autre nom
    nom_fichier_sortie = newfile
    data_frame.to_csv(nom_fichier_sortie, index=False)
    return data_frame

    
  

def regrouper_genres(genre):
    genre = genre.lower()  # Convertir en minuscules pour une correspondance insensible à la casse
    
    if any(keyword in genre for keyword in ['comedy', 'romance']):
        return 'Comedy/Romance'
    elif any(keyword in genre for keyword in ['action', 'thriller', 'suspense']):
        return 'Action/Thriller'
    elif any(keyword in genre for keyword in ['drama', 'melodrama']):
        return 'Drama/Melodrama'
    elif any(keyword in genre for keyword in ['sci-fi', 'science fiction', 'fantasy']):
        return 'Sci-Fi/Fantasy'
    elif 'horror' in genre:
        return 'Horror'
    else:
        return 'Other'

models = {
    'SVM': SVC(class_weight='balanced'),
    'Random Forest': RandomForestClassifier(class_weight='balanced'),
    'Multinomial Naive Bayes': MultinomialNB(),
    'Perceptron': Perceptron(class_weight='balanced')
}

def printresult(model_name,accuracy,classification_report_result):
    print(f'\nResults for {model_name}:')
    print(f'Accuracy: {accuracy}')
    print('Classification Report:')
    print(classification_report_result)
    print('-' * 50)


def evaluation(predictions,y_test):
    accuracy = accuracy_score(y_test, predictions)
    classification_report_result = classification_report(y_test, predictions)
    return accuracy, classification_report_result

def lemmatiser(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

PREMIERE MISE EN FORME DES DONNEES

In [ ]:

#Movie datasets
datamovie = preprocess_comma("movieplot.csv","datamovie.csv")
#REDUCTION DU DATAFRAME CAR TROP D'INSTANCES 

#ON CHOISI LE NOMBRE D INSTANCES
datamovie= datamovie.sample(n=1000, random_state=7)







In [ ]:

# Appliquer la fonction sur la colonne 'Genre' pour créer une nouvelle colonne 'Genre Regroupe'
datamovie['Genre'] = datamovie['Genre'].apply(regrouper_genres)
#TROP DE CLASSES DONC ON ENLEVE OTHER
datamovie = datamovie[datamovie['Genre'] != 'Other']
# Réinitialiser les index après la suppression des lignes
datamovie.reset_index(drop=True, inplace=True)

X = datamovie['Plot']
y = datamovie['Genre']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

for model_name, model in models.items():
    print(f"Training {model_name}...")
    pipeline1 = make_pipeline(CountVectorizer(), model)
    pipeline2 = make_pipeline(TfidfVectorizer(), model)
# Entraîner le modèle
    pipeline1.fit(X_train, y_train)
    pipeline2.fit(X_train, y_train)
# Prédictions sur l'ensemble de test
    predictions1 = pipeline1.predict(X_test)
    predictions2 = pipeline2.predict(X_test)
    
# Évaluation des performances du modèle avec COUNTVECTORIZER 
    accuracy1 , classification_report_result1 = evaluation(predictions1,y_test)
    accuracy2 , classification_report_result2 = evaluation(predictions2,y_test)
    # Affichage
    print("-----------------COUNTVECTORIZER------------------")
    printresult(model_name,accuracy1,classification_report_result1)
    print("---------------------TFIDF------------------------")
    printresult(model_name,accuracy2,classification_report_result2)
    


ON VA SURECHANTILLONNER LES CLASSES SOUS REPRESENTEES

In [ ]:
X = datamovie['Plot']
y = datamovie['Genre']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# Vectoriseurs à tester
vectorizers = {
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer': TfidfVectorizer()
}

for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())        

LES N-GRAM POUR PLUS DE CONTEXTE

UNI ET BI GRAM

In [ ]:
X = datamovie['Plot']
y = datamovie['Genre']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# ------------------------Vectoriseurs à tester + ON RAJOUTE LES NGRAM C'EST LA SEULE DIFFERENCE DANS CETTE CELLULE-------------
vectorizers = {
    'CountVectorizer': CountVectorizer(ngram_range=(1, 2)),
    'TfidfVectorizer': TfidfVectorizer(ngram_range=(1, 2))
}
#~-----------------------------------------------------------------------------------------------------------------------------
for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())        

UNI BI ET TRI

In [ ]:
X = datamovie['Plot']
y = datamovie['Genre']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# ------------------------Vectoriseurs à tester + ON RAJOUTE LES NGRAM C'EST LA SEULE DIFFERENCE DANS CETTE CELLULE-------------
vectorizers = {
    'CountVectorizer': CountVectorizer(ngram_range=(1, 3)),
    'TfidfVectorizer': TfidfVectorizer(ngram_range=(1, 3))
}
#~-----------------------------------------------------------------------------------------------------------------------------
for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())        

ENLEVER STOP WORDS ?

In [ ]:
# ON VA UTILISER LES STOPSWORDS DE NLTK
stop_words = set(stopwords.words('english'))

X = datamovie['Plot']
y = datamovie['Genre']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

#-------------------- Vectoriseurs à tester + ON PEUX METTRE LES STOPWORDS EN ARGUMENT DU VECTORISEUR ----------------------------------
vectorizers = {
    'CountVectorizer': CountVectorizer(stop_words=stop_words),
    'TfidfVectorizer': TfidfVectorizer(stop_words=stop_words)
}
#--------------------------------------------------------------
for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())   

LEMMATISER ?

In [ ]:

X = datamovie['Plot']
#---------------------C'EST ICI QU'ON LEMMATISE AVEC WORD NET ---------------------------
X = X.apply(lemmatiser)
#----------------------------------------------------------------------------------------

y = datamovie['Genre']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# Vectoriseurs à tester
vectorizers = {
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer': TfidfVectorizer()
}

for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())        

MODIFIER TAILLE APPRENTISSAGE ET TAILLE TEST

0.3 -> 0,2

In [ ]:

X = datamovie['Plot']
#---------------------C'EST ICI QU'ON LEMMATISE AVEC WORD NET ---------------------------
X = X.apply(lemmatiser)
#----------------------------------------------------------------------------------------

y = datamovie['Genre']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Vectoriseurs à tester
vectorizers = {
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer': TfidfVectorizer()
}

for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())        

0.2 -> 0.4

In [ ]:

X = datamovie['Plot']
#---------------------C'EST ICI QU'ON LEMMATISE AVEC WORD NET ---------------------------
X = X.apply(lemmatiser)
#----------------------------------------------------------------------------------------

y = datamovie['Genre']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=2)

# Vectoriseurs à tester
vectorizers = {
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer': TfidfVectorizer()
}

for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())        

AUGMENTER LES DONNEES ? 1000 -> 4000

In [ ]:



#Movie datasets
datamovie = preprocess_comma("movieplot.csv","datamovie.csv")
#REDUCTION DU DATAFRAME CAR TROP D'INSTANCES 

#ON CHOISI LE NOMBRE D INSTANCES ici 4000
datamovie= datamovie.sample(n=4000, random_state=7)
# Appliquer la fonction sur la colonne 'Genre' pour créer une nouvelle colonne 'Genre Regroupe'
datamovie['Genre'] = datamovie['Genre'].apply(regrouper_genres)
#TROP DE CLASSES DONC ON ENLEVE OTHER
datamovie = datamovie[datamovie['Genre'] != 'Other']
# Réinitialiser les index après la suppression des lignes
datamovie.reset_index(drop=True, inplace=True)




X = datamovie['Plot']
#---------------------C'EST ICI QU'ON LEMMATISE AVEC WORD NET ---------------------------
X = X.apply(lemmatiser)
#----------------------------------------------------------------------------------------
y = datamovie['Genre']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Vectoriseurs à tester
vectorizers = {
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer': TfidfVectorizer()
}

for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
print(y_train.value_counts())
print(y_train_resampled.value_counts())      
  

1000 -> 10000

In [ ]:



#Movie datasets
datamovie = preprocess_comma("movieplot.csv","datamovie.csv")
#REDUCTION DU DATAFRAME CAR TROP D'INSTANCES 

#ON CHOISI LE NOMBRE D INSTANCES ici 10000
datamovie= datamovie.sample(n=10000, random_state=7)
# Appliquer la fonction sur la colonne 'Genre' pour créer une nouvelle colonne 'Genre Regroupe'
datamovie['Genre'] = datamovie['Genre'].apply(regrouper_genres)
#TROP DE CLASSES DONC ON ENLEVE OTHER
datamovie = datamovie[datamovie['Genre'] != 'Other']
# Réinitialiser les index après la suppression des lignes
datamovie.reset_index(drop=True, inplace=True)




X = datamovie['Plot']
#---------------------C'EST ICI QU'ON LEMMATISE AVEC WORD NET ---------------------------
X = X.apply(lemmatiser)
#----------------------------------------------------------------------------------------
y = datamovie['Genre']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# Vectoriseurs à tester
vectorizers = {
    'CountVectorizer': CountVectorizer(),
    'TfidfVectorizer': TfidfVectorizer()
}

for vectorizer_name, vectorizer in vectorizers.items():
    # Appliquer le suréchantillonnage aléatoire après la vectorisation
    X_train_vectorized = vectorizer.fit_transform(X_train)
    
    ros = RandomOverSampler(random_state=42)
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train_vectorized, y_train)
   
    for model_name, model in models.items():
        print(f"Training {model_name} with {vectorizer_name}...")
        
        # Créer le pipeline avec le modèle choisi
        model_pipeline = make_pipeline(model)
        
        # Entraîner le modèle sur les données resamplées
        model_pipeline.fit(X_train_resampled, y_train_resampled)

        # Prédictions sur l'ensemble de test
        predictions = model_pipeline.predict(vectorizer.transform(X_test))
        
        # Évaluation des performances du modèle
        accuracy, classification_report_result = evaluation(predictions, y_test)
        
        # Affichage
        print(f"-----------------{model_name} with {vectorizer_name}------------------")
        printresult(model_name, accuracy, classification_report_result)
        
print(y_train.value_counts())
print(y_train_resampled.value_counts())      
  